In [1]:
import pandas as pd
import numpy as np
import pickle
from tools import *

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
with open("models/vt.pkl","rb") as f:
    vt = pickle.load(f)
with open("models/std.pkl","rb") as f:
    std = pickle.load(f)
with open("models/discretizer.pkl","rb") as f:
    discretizer = pickle.load(f)

In [4]:
model_names = ["dt", "gnb", "lr", "mnb", "mygnb", "mymnb", "lgbm"]
model_type = [0,0,0,1,0,1,0]
models = {}

for model_name in model_names:
    with open(f"models/{model_name}.pkl", "rb") as f:
        models[model_name] = pickle.load(f)

In [5]:
res = pd.DataFrame()
for model in model_names:
    res[model + " acc"]=None
    res[model + " auc"]=None
    res[model + " f1"]=None
res['dataset']=None
row=0

In [ ]:
data = ["cic-bot-iot","cic-ids-collection","cic-ton-iot","cupid-2022","ddos-dataset","sdn-intrusion","usb-ids-1"]
for name in data:
    df_name = f"datasets/{name}/Test.parquet"
    print(df_name)
    res.loc[row, 'dataset'] = name
    X_cont,y = prepare_data(df_name,vt,std)
    X_disc,_ = prepare_data(df_name,vt,discretizer)
    _,count = np.unique(y,return_counts = True)
    print(X_cont.shape,y.shape,count)
    for model_name,model in models.items():
        if model_type[model_names.index(model_name)] == 0:
            y_prob = model.predict_proba(X_cont)
            y_score = y_prob[:,1]
            y_pred = np.argmax(y_prob,axis=1)
        else:
            y_prob = model.predict_proba(X_disc)
            y_score = y_prob[:,1]
            y_pred = np.argmax(y_prob,axis=1)
        name,acc,f1,auc = print_metrics(model_name,y,y_pred,y_score)
        res.loc[row, model_name + " acc"] = acc
        res.loc[row, model_name + " auc"] = auc
        res.loc[row, model_name + " f1"] = f1
    row+=1

In [ ]:
data = ["cic-bot-iot","cic-ids-collection","cic-ton-iot","cupid-2022","ddos-dataset","sdn-intrusion","usb-ids-1"]
for name in data:
    df_name = f"datasets/{name}/Test.parquet"
    print(df_name)
    res.loc[row, 'dataset'] = name + " balanced"
    X_cont,y = prepare_data_balanced(df_name,vt,std)
    X_disc,_ = prepare_data_balanced(df_name,vt,discretizer)
    print(X_cont.shape,y.shape)
    for model_name,model in models.items():
        if model_type[model_names.index(model_name)] == 0:
            y_prob = model.predict_proba(X_cont)
            y_score = y_prob[:,1]
            y_pred = np.argmax(y_prob,axis=1)
        else:
            y_prob = model.predict_proba(X_disc)
            y_score = y_prob[:,1]
            y_pred = np.argmax(y_prob,axis=1)
        name,acc,f1,auc = print_metrics(model_name,y,y_pred,y_score)
        res.loc[row, model_name + " acc"] = acc
        res.loc[row, model_name + " auc"] = auc
        res.loc[row, model_name + " f1"] = f1
    row+=1

In [ ]:
res.to_excel("fig/res2.xlsx")